In [3]:
import numpy as np;
import pandas as pd;

## Initialize Parameters

In [4]:
## parameters and global variables

district = "tvm"

jC = 0.3
jM = 0.2
jP = 0.1

muC = 0.05
muM = 0.1
muP = 0.1

## Load and Initialize Data

In [6]:
## Load population data
initDataDF = pd.read_csv("../data/" + district + "_init_data.csv")

## Load distance data
distanceData = pd.read_csv("../data/" + district + "_distance_matrix.csv")

## Number of regions
r = len(initDataDF.index)

## Sort population data and distance data by name for easy matrix transformation
initDataDF.sort_values(by=['name'], inplace=True)
distanceData.sort_values(by=['fromName', 'toName'], inplace=True)

## Job column
initDataDF.loc[initDataDF['type']=='C','J'] = initDataDF[initDataDF['type']=='C']['S']*jC
initDataDF.loc[initDataDF['type']=='M','J'] = initDataDF[initDataDF['type']=='M']['S']*jM
initDataDF.loc[initDataDF['type']=='P','J'] = initDataDF[initDataDF['type']=='P']['S']*jP

initDataDF.loc[initDataDF['type']=='C','T'] = initDataDF[initDataDF['type']=='C']['S']*muC
initDataDF.loc[initDataDF['type']=='M','T'] = initDataDF[initDataDF['type']=='M']['S']*muM
initDataDF.loc[initDataDF['type']=='P','T'] = initDataDF[initDataDF['type']=='P']['S']*muP

initDataDF.tail(30)

,name,type,S,E,I,H,R,J,T
47,Ottasekharamangalam__Thiruvananthapuram,P,18794,0,0,0,0,1879.4,1879.4
48,Ottoor__Thiruvananthapuram,P,16085,0,0,0,0,1608.5,1608.5
49,Pallichal__Thiruvananthapuram,P,45219,0,0,0,0,4521.9,4521.9
50,Pallikkal__Thiruvananthapuram,P,16900,0,0,0,0,1690.0,1690.0
51,Panavoor__Thiruvananthapuram,P,20348,0,0,0,0,2034.8,2034.8
52,Pangod__Thiruvananthapuram,P,29039,0,0,0,0,2903.9,2903.9
53,Parassala__Thiruvananthapuram,P,52263,0,0,0,0,5226.3,5226.3
54,Pazhayakunnummel__Thiruvananthapuram,P,24608,0,0,0,0,2460.8,2460.8
55,Peringammala__Thiruvananthapuram,P,27667,0,0,0,0,2766.7,2766.7
56,Perumkadavila__Thiruvananthapuram,P,23385,0,0,0,0,2338.5,2338.5


## Prepare Distance matrix

In [4]:
distanceMatrix = distanceData['value'].to_numpy().reshape(r,r)
#distanceMatrix[0][4]

## Function Definitions

In [5]:
def getSij(i, j, distanceMatrix, initDataDF):
    #print(distanceMatrix[i] <= distanceMatrix[i][j])
    return(initDataDF[distanceMatrix[i] <= distanceMatrix[i][j]]['J'].sum() - initDataDF.iloc[i]['J'] - initDataDF.iloc[j]['J'])
    
#getSij(63,23,distanceMatrix, initDataDF)

In [6]:
def getTij(i,j, distanceMatrix, initDataDF):
    Ni = initDataDF.iloc[i]['S']
    Ti = initDataDF.iloc[i]['T']

    if(i==j):
        return Ni - Ti;
    
    Ji = initDataDF.iloc[i]['J']
    Jj = initDataDF.iloc[j]['J']

    Sij = getSij(i,j,distanceMatrix, initDataDF)
    Tij = Ti * ((Ji * Jj) /((Ji + Sij)*(Ji + Jj + Sij)))
    return Tij
    
    

## Compute TMatrix

In [7]:
data = []

for i in range(0,r):
    for j in range(0,r):
        Tij = getTij(i,j,distanceMatrix, initDataDF)
        #print("(", i, ",", j, ") ",  initDataDF.iloc[i]['name'], " ", initDataDF.iloc[j]['name'], " " ,Tij)
        data.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Tij': Tij});
        
resultsDF = pd.DataFrame(data);


## Save output

In [8]:
resultsDF.to_csv("../output/" + district + "_tmatrix_results.csv")